In [1]:
import os
import subprocess
from ROOT import TXMLEngine, XMLDocPointer_t, XMLNodePointer_t, Int_t
from ROOT import TFile, TTree
import numpy as np
import pandas as pd

from datetime import datetime
import dateutil

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

Welcome to JupyROOT 6.28/00


In [16]:
def readCalibXML(filename):
    xml = TXMLEngine();
    xmldoc = xml.ParseFile(filename);
    root = xml.DocGetRootElement(xmldoc);
    cmsCondPayload = xml.GetChild(root)
    eb = xml.GetChild(cmsCondPayload)
    ee = xml.GetNext(eb)
    eb_calib = xml.GetChild(eb)
    ee_calib = xml.GetChild(ee)
    eb_abs_calib = []
    cnode = xml.GetChild(eb_calib) # counts
    eb_counts = int(xml.GetNodeContent(cnode)) # number of EB crystals
    cnode = xml.GetNext(cnode)
    cnode = xml.GetNext(cnode)
    i = 0
    for i in range(eb_counts):
        eb_abs_calib.append(float(xml.GetNodeContent(cnode)))
        i += 1
        if i%10000==1: print(i,eb_counts)
        cnode = xml.GetNext(cnode)
    ee_abs_calib = []
    cnode = xml.GetChild(ee_calib) # counts
    ee_counts = int(xml.GetNodeContent(cnode)) # number of EB crystals
    cnode = xml.GetNext(cnode)
    cnode = xml.GetNext(cnode)
    i = 0
    for i in range(ee_counts):
        ee_abs_calib.append(float(xml.GetNodeContent(cnode)))
        i += 1
        if i%10000==1: print(i,ee_counts)
        cnode = xml.GetNext(cnode)
    return np.array(eb_abs_calib),np.array(ee_abs_calib)

In [18]:
def writeCalibXML(filename, eb_abs_calib, ee_abs_calib, eb_global_shift_=0.0, ee_global_shift_=0.0):
    ebclass_begin_txt = '''<?xml version="1.0" encoding="UTF-8" standalone="yes" ?>
    <!DOCTYPE boost_serialization>
    <boost_serialization signature="serialization::archive" version="19">
    <cmsCondPayload class_id="0" tracking_level="0" version="0">
        <eb- class_id="1" tracking_level="0" version="0">
            <m_items>
                <count>61200</count>
                <item_version>0</item_version>
                
    '''
    eeclass_end_txt = '''        </m_items>
        </ee->
    </cmsCondPayload>
    </boost_serialization>
    '''
    ebclass_end_txt = '''        </m_items>
        </eb->
        <ee- class_id="3" tracking_level="0" version="0">
            <m_items>
                <count>14648</count>
                <item_version>0</item_version>
                
    '''
    with open(filename, 'w+') as f0:
        f0.write(ebclass_begin_txt)
        for val in eb_abs_calib:
            f0.write('\t\t\t\t<item>{:0.9E}</item>\n'.format(val-eb_global_shift_))
        f0.write(ebclass_end_txt)
        for val in ee_abs_calib:
            f0.write('\t\t\t\t<item>{:0.9E}</item>\n'.format(val-ee_global_shift_))
        f0.write(eeclass_end_txt)

In [19]:
xmlfile = '/eos/user/b/bjoshi/www/ECAL_timing/Run2023C/Calibration/368331_368384_tCut_10_global_shift/ecalTiming-abs_2023_06_05.xml'
EB, EE = readCalibXML(xmlfile)
newxmlfile = '/eos/user/b/bjoshi/www/ECAL_timing/Run2023C/Calibration/368331_368384_tCut_10_global_shift/ecalTiming-abs_2023_06_30_direct.xml'
writeCalibXML(newxmlfile, EB, EE, eb_global_shift_=2.00, ee_global_shift_=1.0)

1 61200
10001 61200
20001 61200
30001 61200
40001 61200
50001 61200
60001 61200
1 14648
10001 14648
